# Mortgage Lending Exploratory Data Analysis
Analyzing 369K+ mortgage applications to understand approval patterns

In [ ]:
import pandas as pd
import plotly.express as px
from snowflake.snowpark.context import get_active_session

session = get_active_session()

## Load Data

In [ ]:
df = session.table('"COCO-MEETUP-OSLO".PUBLIC.MORTGAGE_LENDING').to_pandas()
print(f"Loaded {len(df):,} records")
df.head()

## Summary Statistics

In [ ]:
df.describe()

In [ ]:
df.info()

## Approval Rate Analysis

In [ ]:
approval_by_type = df.groupby('LOAN_TYPE_NAME').agg(
    total_loans=('LOAN_ID', 'count'),
    approved=('MORTGAGERESPONSE', 'sum'),
    approval_rate=('MORTGAGERESPONSE', 'mean')
).round(3).sort_values('approval_rate', ascending=False)
approval_by_type

In [ ]:
fig = px.bar(approval_by_type.reset_index(), x='LOAN_TYPE_NAME', y='approval_rate',
             title='Approval Rate by Loan Type', color='approval_rate',
             color_continuous_scale='RdYlGn')
fig.update_layout(yaxis_tickformat='.0%')
fig.show()

## Loan Amount Distribution

In [ ]:
fig = px.histogram(df, x='LOAN_AMOUNT_000S', nbins=50, 
                   title='Distribution of Loan Amounts ($000s)',
                   color_discrete_sequence=['steelblue'])
fig.show()

## Top 10 Counties by Volume

In [ ]:
county_stats = df.groupby('COUNTY_NAME').agg(
    total_loans=('LOAN_ID', 'count'),
    avg_loan_amount=('LOAN_AMOUNT_000S', 'mean'),
    approval_rate=('MORTGAGERESPONSE', 'mean')
).round(2).sort_values('total_loans', ascending=False).head(10)
county_stats

In [ ]:
fig = px.bar(county_stats.reset_index(), x='COUNTY_NAME', y='total_loans',
             color='approval_rate', title='Top 10 Counties by Loan Volume',
             color_continuous_scale='RdYlGn')
fig.show()

## Approval Rate by Loan Purpose

In [ ]:
purpose_stats = df.groupby('LOAN_PURPOSE_NAME').agg(
    count=('LOAN_ID', 'count'),
    approval_rate=('MORTGAGERESPONSE', 'mean')
).round(3)

fig = px.pie(purpose_stats.reset_index(), values='count', names='LOAN_PURPOSE_NAME',
             title='Loan Distribution by Purpose')
fig.show()

## Income vs Loan Amount (Approved vs Denied)

In [ ]:
sample = df.dropna(subset=['APPLICANT_INCOME_000S']).sample(min(5000, len(df)))
sample['Status'] = sample['MORTGAGERESPONSE'].map({1: 'Approved', 0: 'Denied'})

fig = px.scatter(sample, x='APPLICANT_INCOME_000S', y='LOAN_AMOUNT_000S', 
                 color='Status', opacity=0.5,
                 title='Income vs Loan Amount by Approval Status',
                 color_discrete_map={'Approved': 'green', 'Denied': 'red'})
fig.show()

## Monthly Trends

In [ ]:
df['MONTH'] = pd.to_datetime(df['TS']).dt.to_period('M').astype(str)
monthly = df.groupby('MONTH').agg(
    volume=('LOAN_ID', 'count'),
    approval_rate=('MORTGAGERESPONSE', 'mean')
).reset_index()

fig = px.line(monthly, x='MONTH', y='volume', title='Monthly Loan Volume',
              markers=True)
fig.show()

# Machine Learning Model
Training a classifier to predict mortgage approval

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import numpy as np

## Data Preparation

In [ ]:
ml_df = df.dropna(subset=['APPLICANT_INCOME_000S', 'LOAN_AMOUNT_000S']).copy()
print(f"Records after dropping nulls: {len(ml_df):,}")

le_loan_type = LabelEncoder()
le_purpose = LabelEncoder()
le_county = LabelEncoder()

ml_df['LOAN_TYPE_ENC'] = le_loan_type.fit_transform(ml_df['LOAN_TYPE_NAME'])
ml_df['PURPOSE_ENC'] = le_purpose.fit_transform(ml_df['LOAN_PURPOSE_NAME'])
ml_df['COUNTY_ENC'] = le_county.fit_transform(ml_df['COUNTY_NAME'])

ml_df['INCOME_TO_LOAN_RATIO'] = ml_df['APPLICANT_INCOME_000S'] / (ml_df['LOAN_AMOUNT_000S'] + 1)

print(f"Features engineered successfully")

## Train/Test Split

In [ ]:
features = ['APPLICANT_INCOME_000S', 'LOAN_AMOUNT_000S', 'LOAN_TYPE_ENC', 
            'PURPOSE_ENC', 'COUNTY_ENC', 'INCOME_TO_LOAN_RATIO']

X = ml_df[features]
y = ml_df['MORTGAGERESPONSE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training samples: {len(X_train):,}")
print(f"Test samples: {len(X_test):,}")
print(f"Approval rate in train: {y_train.mean():.1%}")
print(f"Approval rate in test: {y_test.mean():.1%}")

## Train Random Forest Model

In [ ]:
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=50,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)
print("Model training complete!")

## Model Evaluation

In [ ]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Denied', 'Approved']))

roc_auc = roc_auc_score(y_test, y_prob)
print(f"\nROC-AUC Score: {roc_auc:.4f}")

## Feature Importance

In [ ]:
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=True)

fig = px.bar(importance_df, x='Importance', y='Feature', orientation='h',
             title='Feature Importance for Mortgage Approval Prediction',
             color='Importance', color_continuous_scale='Blues')
fig.show()

## Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=['Actual: Denied', 'Actual: Approved'], 
                     columns=['Pred: Denied', 'Pred: Approved'])

fig = px.imshow(cm, labels=dict(x='Predicted', y='Actual'),
                x=['Denied', 'Approved'], y=['Denied', 'Approved'],
                title='Confusion Matrix', text_auto=True,
                color_continuous_scale='Blues')
fig.show()

print(f"\nModel Summary:")
print(f"- Accuracy: {(y_pred == y_test).mean():.1%}")
print(f"- ROC-AUC: {roc_auc:.3f}")

# XGBoost Model

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)

xgb_model.fit(X_train, y_train)
print("XGBoost training complete!")

In [ ]:
y_pred_xgb = xgb_model.predict(X_test)
y_prob_xgb = xgb_model.predict_proba(X_test)[:, 1]

print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb, target_names=['Denied', 'Approved']))

roc_auc_xgb = roc_auc_score(y_test, y_prob_xgb)
print(f"\nXGBoost ROC-AUC: {roc_auc_xgb:.4f}")
print(f"Random Forest ROC-AUC: {roc_auc:.4f}")
print(f"\nImprovement: {(roc_auc_xgb - roc_auc)*100:.2f}%")

# Register XGBoost Model to Snowflake

In [ ]:
from snowflake.ml.registry import Registry

session.use_database('"COCO-MEETUP-OSLO"')
session.use_schema('PUBLIC')

reg = Registry(session=session, database_name='"COCO-MEETUP-OSLO"', schema_name='PUBLIC')

sample_input = X_train.head(10)

mv = reg.log_model(
    xgb_model,
    model_name="MORTGAGE_APPROVAL_MODEL",
    version_name="V1",
    sample_input_data=sample_input,
    conda_dependencies=["xgboost"],
    comment="XGBoost classifier for mortgage approval prediction. ROC-AUC: 0.744"
)

print(f"Model registered: {mv.model_name} version {mv.version_name}")

# Model Explainability with SHAP

In [ ]:
import shap

explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test.head(1000))

print(f"SHAP values computed for 1,000 samples")

## Feature Importance (SHAP)

In [ ]:
shap_importance = pd.DataFrame({
    'Feature': features,
    'Mean_SHAP': abs(shap_values).mean(axis=0)
}).sort_values('Mean_SHAP', ascending=True)

fig = px.bar(shap_importance, x='Mean_SHAP', y='Feature', orientation='h',
             title='SHAP Feature Importance (Mean |SHAP|)',
             color='Mean_SHAP', color_continuous_scale='Reds')
fig.show()

## SHAP Summary Plot

In [ ]:
shap.summary_plot(shap_values, X_test.head(1000), feature_names=features, show=False)
import matplotlib.pyplot as plt
plt.tight_layout()
plt.show()

## Single Prediction Explanation

In [ ]:
sample_idx = 0
sample = X_test.iloc[[sample_idx]]
prediction = xgb_model.predict(sample)[0]
proba = xgb_model.predict_proba(sample)[0][1]

print(f"Sample prediction: {'Approved' if prediction == 1 else 'Denied'}")
print(f"Approval probability: {proba:.1%}")
print(f"\nFeature contributions:")

for feat, val, shap_val in zip(features, sample.values[0], shap_values[sample_idx]):
    direction = '↑' if shap_val > 0 else '↓'
    print(f"  {feat}: {val:.2f} → SHAP: {shap_val:+.3f} {direction}")

# Create Predictions Table for Model Monitoring

In [ ]:
predictions_df = X_test.copy()
predictions_df['PREDICTION'] = xgb_model.predict(X_test)
predictions_df['PREDICTION_PROBA'] = xgb_model.predict_proba(X_test)[:, 1]
predictions_df['ACTUAL'] = y_test.values
predictions_df['PREDICTION_TS'] = pd.Timestamp.now()

snowpark_df = session.create_dataframe(predictions_df)
snowpark_df.write.mode('overwrite').save_as_table('"COCO-MEETUP-OSLO".PUBLIC.MORTGAGE_PREDICTIONS')

print(f"Predictions table created with {len(predictions_df):,} rows")

# Set Up Model Monitoring

In [ ]:
monitor_sql = '''
CREATE OR REPLACE VIEW "COCO-MEETUP-OSLO".PUBLIC.MODEL_MONITORING_DASHBOARD AS
SELECT
    DATE_TRUNC('hour', PREDICTION_TS) AS HOUR,
    COUNT(*) AS PREDICTION_COUNT,
    SUM(CASE WHEN PREDICTION = ACTUAL THEN 1 ELSE 0 END) AS CORRECT_PREDICTIONS,
    ROUND(AVG(CASE WHEN PREDICTION = ACTUAL THEN 1 ELSE 0 END), 4) AS ACCURACY,
    ROUND(AVG(PREDICTION_PROBA), 4) AS AVG_CONFIDENCE,
    ROUND(STDDEV(PREDICTION_PROBA), 4) AS CONFIDENCE_STDDEV,
    SUM(PREDICTION) AS PREDICTED_APPROVALS,
    SUM(ACTUAL) AS ACTUAL_APPROVALS,
    ROUND(AVG(APPLICANT_INCOME_000S), 2) AS AVG_INCOME,
    ROUND(AVG(LOAN_AMOUNT_000S), 2) AS AVG_LOAN_AMOUNT
FROM "COCO-MEETUP-OSLO".PUBLIC.MORTGAGE_PREDICTIONS
GROUP BY 1
ORDER BY 1
'''

session.sql(monitor_sql).collect()
print("Monitoring dashboard view created!")

metrics = session.sql('SELECT * FROM "COCO-MEETUP-OSLO".PUBLIC.MODEL_MONITORING_DASHBOARD').to_pandas()
metrics

## Get Monitoring Metrics

In [ ]:
print("Overall Model Performance:")
print(f"  Total predictions: {metrics['PREDICTION_COUNT'].sum():,}")
print(f"  Overall accuracy: {metrics['ACCURACY'].mean():.1%}")
print(f"  Avg confidence: {metrics['AVG_CONFIDENCE'].mean():.1%}")
print(f"  Predicted approvals: {metrics['PREDICTED_APPROVALS'].sum():,}")
print(f"  Actual approvals: {metrics['ACTUAL_APPROVALS'].sum():,}")

# SQL Inference with Registered Model

In [ ]:
mv_warehouse = reg.log_model(
    xgb_model,
    model_name="MORTGAGE_APPROVAL_MODEL",
    version_name="V2_WAREHOUSE",
    sample_input_data=X_train.head(10).reset_index(drop=True),
    target_platforms=["WAREHOUSE"],
    comment="XGBoost classifier - warehouse inference enabled"
)

print(f"Model registered: {mv_warehouse.model_name} version {mv_warehouse.version_name}")

## Run SQL Inference on New Data

In [ ]:
test_data = session.create_dataframe(X_test.head(10).reset_index(drop=True))

predictions = mv_warehouse.run(test_data, function_name='predict')
print("Inference Results:")
predictions.to_pandas()

## Batch Inference on Full Table

In [ ]:
encoded_data_sql = '''
SELECT 
    LOAN_ID,
    LOAN_TYPE_NAME,
    LOAN_PURPOSE_NAME,
    COUNTY_NAME,
    APPLICANT_INCOME_000S,
    LOAN_AMOUNT_000S,
    CASE LOAN_TYPE_NAME 
        WHEN 'Conventional' THEN 0
        WHEN 'FHA-insured' THEN 1
        WHEN 'FSA/RHS-guaranteed' THEN 2
        WHEN 'VA-guaranteed' THEN 3
    END AS LOAN_TYPE_ENC,
    CASE LOAN_PURPOSE_NAME
        WHEN 'Home improvement' THEN 0
        WHEN 'Home purchase' THEN 1
        WHEN 'Refinancing' THEN 2
    END AS PURPOSE_ENC,
    MOD(ABS(HASH(COUNTY_NAME)), 63) AS COUNTY_ENC,
    APPLICANT_INCOME_000S / (LOAN_AMOUNT_000S + 1) AS INCOME_TO_LOAN_RATIO
FROM "COCO-MEETUP-OSLO".PUBLIC.MORTGAGE_LENDING
WHERE APPLICANT_INCOME_000S IS NOT NULL
LIMIT 10
'''

batch_data = session.sql(encoded_data_sql)

feature_cols = ['APPLICANT_INCOME_000S', 'LOAN_AMOUNT_000S', 'LOAN_TYPE_ENC', 'PURPOSE_ENC', 'COUNTY_ENC', 'INCOME_TO_LOAN_RATIO']
features_df = batch_data.select(feature_cols)

predictions = mv_warehouse.run(features_df, function_name='predict')
pred_df = predictions.to_pandas()
print("Prediction columns:", pred_df.columns.tolist())

result = batch_data.to_pandas()
result['PREDICTED_APPROVAL'] = pred_df.iloc[:, -1].map({0: 'Denied', 1: 'Approved'})

print("\nBatch Inference from MORTGAGE_LENDING Table:")
result[['LOAN_ID', 'LOAN_TYPE_NAME', 'COUNTY_NAME', 'APPLICANT_INCOME_000S', 'LOAN_AMOUNT_000S', 'PREDICTED_APPROVAL']]

# Prediction Performance Analysis by Segment

In [ ]:
full_analysis_sql = '''
SELECT 
    LOAN_ID,
    LOAN_TYPE_NAME,
    LOAN_PURPOSE_NAME,
    COUNTY_NAME,
    APPLICANT_INCOME_000S,
    LOAN_AMOUNT_000S,
    MORTGAGERESPONSE AS ACTUAL,
    CASE LOAN_TYPE_NAME 
        WHEN 'Conventional' THEN 0
        WHEN 'FHA-insured' THEN 1
        WHEN 'FSA/RHS-guaranteed' THEN 2
        WHEN 'VA-guaranteed' THEN 3
    END AS LOAN_TYPE_ENC,
    CASE LOAN_PURPOSE_NAME
        WHEN 'Home improvement' THEN 0
        WHEN 'Home purchase' THEN 1
        WHEN 'Refinancing' THEN 2
    END AS PURPOSE_ENC,
    MOD(ABS(HASH(COUNTY_NAME)), 63) AS COUNTY_ENC,
    APPLICANT_INCOME_000S / (LOAN_AMOUNT_000S + 1) AS INCOME_TO_LOAN_RATIO
FROM "COCO-MEETUP-OSLO".PUBLIC.MORTGAGE_LENDING
WHERE APPLICANT_INCOME_000S IS NOT NULL
'''

full_data = session.sql(full_analysis_sql)
feature_cols = ['APPLICANT_INCOME_000S', 'LOAN_AMOUNT_000S', 'LOAN_TYPE_ENC', 'PURPOSE_ENC', 'COUNTY_ENC', 'INCOME_TO_LOAN_RATIO']

predictions = mv_warehouse.run(full_data.select(feature_cols), function_name='predict')

analysis_df = full_data.to_pandas()
analysis_df['PREDICTED'] = predictions.to_pandas()['output_feature_0']
print(f"Analyzed {len(analysis_df):,} loans")

## Performance by Loan Type

In [ ]:
loan_type_perf = analysis_df.groupby('LOAN_TYPE_NAME').agg(
    total_loans=('LOAN_ID', 'count'),
    actual_approval_rate=('ACTUAL', 'mean'),
    predicted_approval_rate=('PREDICTED', 'mean'),
    accuracy=('LOAN_ID', lambda x: (analysis_df.loc[x.index, 'ACTUAL'] == analysis_df.loc[x.index, 'PREDICTED']).mean())
).round(3).sort_values('actual_approval_rate', ascending=False)

loan_type_perf['rate_difference'] = loan_type_perf['predicted_approval_rate'] - loan_type_perf['actual_approval_rate']
print("Performance by Loan Type:")
loan_type_perf

In [ ]:
fig = px.bar(loan_type_perf.reset_index(), x='LOAN_TYPE_NAME', 
             y=['actual_approval_rate', 'predicted_approval_rate'],
             barmode='group', title='Actual vs Predicted Approval Rates by Loan Type',
             labels={'value': 'Approval Rate', 'variable': 'Rate Type'})
fig.show()

## Top 15 Counties by Volume with Performance

In [ ]:
county_perf = analysis_df.groupby('COUNTY_NAME').agg(
    total_loans=('LOAN_ID', 'count'),
    actual_approval_rate=('ACTUAL', 'mean'),
    predicted_approval_rate=('PREDICTED', 'mean'),
    correct_predictions=('LOAN_ID', lambda x: (analysis_df.loc[x.index, 'ACTUAL'] == analysis_df.loc[x.index, 'PREDICTED']).sum()),
    avg_income=('APPLICANT_INCOME_000S', 'mean'),
    avg_loan=('LOAN_AMOUNT_000S', 'mean')
).round(3)

county_perf['accuracy'] = (county_perf['correct_predictions'] / county_perf['total_loans']).round(3)
top_counties = county_perf.sort_values('total_loans', ascending=False).head(15)

print("Top 15 Counties by Volume:")
top_counties[['total_loans', 'actual_approval_rate', 'predicted_approval_rate', 'accuracy', 'avg_income', 'avg_loan']]

In [ ]:
fig = px.scatter(top_counties.reset_index(), x='actual_approval_rate', y='predicted_approval_rate',
                 size='total_loans', color='accuracy', hover_name='COUNTY_NAME',
                 title='Actual vs Predicted Approval Rate by County (size = volume)',
                 color_continuous_scale='RdYlGn')
fig.add_shape(type='line', x0=0.6, y0=0.6, x1=1, y1=1, line=dict(dash='dash', color='gray'))
fig.show()

## Highest & Lowest Approval Rate Segments

In [ ]:
print("=" * 60)
print("HIGHEST ACTUAL APPROVAL RATES (min 1000 loans):")
print("=" * 60)
high_approval = county_perf[county_perf['total_loans'] >= 1000].sort_values('actual_approval_rate', ascending=False).head(10)
print(high_approval[['total_loans', 'actual_approval_rate', 'accuracy']].to_string())

print("\n" + "=" * 60)
print("LOWEST ACTUAL APPROVAL RATES (min 1000 loans):")
print("=" * 60)
low_approval = county_perf[county_perf['total_loans'] >= 1000].sort_values('actual_approval_rate').head(10)
print(low_approval[['total_loans', 'actual_approval_rate', 'accuracy']].to_string())

## Model Accuracy Heatmap: Loan Type x Purpose

In [ ]:
segment_perf = analysis_df.groupby(['LOAN_TYPE_NAME', 'LOAN_PURPOSE_NAME']).agg(
    count=('LOAN_ID', 'count'),
    accuracy=('LOAN_ID', lambda x: (analysis_df.loc[x.index, 'ACTUAL'] == analysis_df.loc[x.index, 'PREDICTED']).mean()),
    approval_rate=('ACTUAL', 'mean')
).round(3).reset_index()

pivot_accuracy = segment_perf.pivot(index='LOAN_TYPE_NAME', columns='LOAN_PURPOSE_NAME', values='accuracy')

fig = px.imshow(pivot_accuracy, text_auto='.1%', 
                title='Model Accuracy by Loan Type & Purpose',
                color_continuous_scale='RdYlGn', zmin=0.6, zmax=0.9)
fig.show()

print("\nSegment Details:")
segment_perf.sort_values('accuracy', ascending=False)